### User input

In [ ]:
decimal = '.'                                                                   # decimal of the input file
sep = ';'                                                                       # separator of the input file

from config import *                                                            # Personal settings of local user to set input and output directories
input_dir = input_directory + 'results/df_mediancor_sw_30.0.csv'                # input directory
output = output_directory + 'results/'                                          # output directory

dateheadername = 'Timestamp'                                                    # header of the date  
header = 0                                                                      # header row number
sample_name = 'sw'                                                              # name of the sample

# To get a kde plot, the user can modify:
fig_format = '.tiff'                                                            # format of the exported figure
dpi = 300                                                                       # resolution of the exported figure

### Start environment and import data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import seaborn as sns

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)               # make sure time column (here index) is using time format
df

### kdeplot

In [ ]:
def makeakdeplot(df_in, 
                 output_dir,
                 label_name,
                 col_sel = None, 
                 xlabel = None, 
                 ylabel = None, 
                 yminlim = None, 
                 ymaxlim = None, 
                 ymajlocator = None, 
                 yminlocator = None):                  
    '''
    function to get a kde plot
    :argument df_in: dataframe in input
    :argument output_dir: directory where storing the results
    :argument label_name: sample name
    :argument col_sel: selected wavelength columns
    :argument xlabel, ylabel: x- and y-axes label
    :argument yminlim, ymaxlim: y-axis limits
    :argument yminlocator, ymajlocator: y-axis major and minor ticks 
    :return: gaussian kde plot at specific wavelengths
    '''
    factor = .5
    plot_width = 25*factor # define the plot dimensions in inches (width and height)
    plot_height = 13*factor
        
    if col_sel is None:
        col_sel = ['255 nm','275 nm','295 nm','350 nm','400 nm','697.5 nm']
    for i in col_sel:
        sns.kdeplot(df_in[i], kernel='gau') 
       
    if xlabel is None:
        xlabel = 'Absorbance [abs m$^{-1}$]'
    if  ylabel is None:
        ylabel = 'Gaussian KDE [dimesionless]'        
    if yminlim is None:
        ymin = (df_in[col_sel].min()).min()                                         
        yminlim = ymin - 0.05*abs(ymin)
    if ymaxlim is None:
        ymax = (df_in[col_sel].max()).max()
        ymaxlim = ymax + 0.05*abs(ymax) 
    if ymajlocator is None:
        ymajlocator = round((ymaxlim-yminlim)/6,2) 
    if yminlocator is None:
        yminlocator = ymajlocator/3

    plt.style.use ('tableau-colorblind10')                                           # use colors colorblind friendly
    plt.xlabel(xlabel)                                                               # label the x-axis 
    plt.ylabel(ylabel)                                                               # label the y-axis 
    plt.rc('font', family='Arial')                                                   # define the font
    plt.rcParams['axes.labelsize'] = 20                                              # define the label size
    #plt.rcParams['axes.labelweight'] = 'bold'                                       # define the label weight
    plt.gca().tick_params(axis = 'both', which = 'major', length = 5, labelsize = 18)# define the label size of the labels on the axes-ticks (major and minor) and length and color of ticks 
    plt.gca().tick_params(which = 'minor', length = 4, colors = 'k')
    #plt.minorticks_on()                                                              # turn on the minor ticks on both the axes
    plt.gca().set_ylim(yminlim,ymaxlim)                                               # define the y-axis range
    plt.gca().yaxis.set_major_locator(MultipleLocator(ymajlocator))                   # define the y-axes major and minor ticks 
    plt.gca().yaxis.set_minor_locator(MultipleLocator(yminlocator))                     

    #plt.grid(which='major', linestyle=':', linewidth='1', color='gray')              # define the major grid
    #plt.grid(which='minor', linestyle=':', axis = 'x', linewidth='1', color='gray')  # define the minor grid

    plt.legend(loc='best', fontsize = 17, markerscale = 2)                            # define the legend 
                                                                                      
    plt.rcParams['figure.figsize'] = (plot_width, plot_height)                        # define the size of the figure
    #plt.tight_layout() 
    fig = plt.gca().get_figure()                                                      # return the figure 
    fig.savefig(output_dir + 'kde_' + str(label_name) + fig_format, dpi = dpi)        # save the figure to the desired format and resolution

In [ ]:
#%matplotlib inline # necessary if the notebook is not configured to use the inline backend by default
%matplotlib notebook
plt.ion()
makeakdeplot(df, output, sample_name)